<div style="background-color: #FFDDDD; border-left: 5px solid red; padding: 10px; color: black;">
    <strong>Kernel:</strong> Python 3 (ipykernel)
</div>

# 🚀 Deploy `deepseek-ai/DeepSeek-R1-Distill-Llama-8B` on Amazon SageMaker

To start off, let's install some packages to help us through the notebooks. **Restart the kernel after packages have been installed.**

In [ ]:
%pip install -r ./scripts/requirements.txt --upgrade

In [ ]:
import os
import sagemaker
from sagemaker.djl_inference import DJLModel
from ipywidgets import Dropdown

import sys
sys.path.append(os.path.dirname(os.getcwd()))

from utilities.helpers import (
    pretty_print_html, 
    set_meta_llama_params,
    print_dialog,
    format_messages,
    write_eula
)

In [ ]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = session.default_bucket()

print(f"Execution Role: {role}")
print(f"Default S3 Bucket: {default_bucket}")

## Deploy Model to SageMaker Hosting

### Step 1: Get SageMaker LMI Container to host DeepSeek

In [ ]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="djl-lmi", 
    region=session.boto_session.region_name, 
    version="0.29.0"
)
pretty_print_html(f"using image to host: {inference_image_uri}")

### Step 2: Deploy model using `DJLModel`

In [12]:
inference_llm_config = {
    "HF_MODEL_ID": "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    "OPTION_MAX_MODEL_LEN": "4096",
    "OPTION_GPU_MEMORY_UTILIZATION": "0.8",
    "OPTION_ENABLE_STREAMING": "false",
    "OPTION_ROLLING_BATCH": "auto",
    "OPTION_MODEL_LOADING_TIMEOUT": "3600",
    # "OPTION_OUTPUT_FORMATTER": "jsonlines",
    "OPTION_PAGED_ATTENTION": "false",
    "OPTION_DTYPE": "fp16",
}

In [13]:
model_name = "DeepSeek-R1-Distill-Llama-8B"

lmi_model = sagemaker.Model(
    image_uri=inference_image_uri,
    env=inference_llm_config,
    role=role,
    name=model_name
)

In [ ]:
base_endpoint_name = f"{model_name}-endpoint"

predictor = lmi_model.deploy(
    initial_instance_count=1, 
    instance_type="ml.g5.2xlarge",
    endpoint_name=base_endpoint_name
)